In [4]:
##### 기본 정보 입력 #####
import streamlit as st
from openai import OpenAI
import uuid
import os
import import_ipynb

# GPT-4와 Dalle 호출하는 함수
from ch09_gpt import get_llm
from ch09_dalle import get_image_by_dalle

st.set_page_config(page_title='📚NovelGPT', layout='wide', initial_sidebar_state='expanded')

##### 기능 구현 함수 #####

# 스토리 & 이미지 캐싱
@st.cache_resource(show_spinner='Generating your story...')
def get_story_and_image(genre, user_choice, api_key):
    client = OpenAI(api_key=api_key)  # 환경 변수 직접 수정 대신 객체 생성 방식 사용
    llm_model = get_llm("test")

    # 사용자의 선택을 기반으로 LLM 호출
    llm_result = llm_model.invoke({"input": user_choice}, config={"configurable": {"session_id": "test"}}).content

    response_list = llm_result.split("\n")
    
    # 선택지가 없을 경우 대비
    if len(response_list) < 2:
        return {'story': llm_result, 'decisionQuestion': None, 'choices': None, 'dalle_img': None}

    # DALL·E 프롬프트 추출
    img_prompt = response_list[-1] if "Dalle Prompt" in response_list[-1] else None
    dalle_img = get_image_by_dalle(client, genre, img_prompt) if img_prompt else None

    # 불필요한 부분 제거
    responses = [s.strip() for s in response_list if s.strip() and '-- -- --' not in s]
    responses = [s for s in responses if 'Dalle Prompt' not in s and 'Image prompt' not in s]

    # 데이터 정리
    story = ''
    choices = []
    decisionQuestion = None

    for response in responses:
        if response.startswith('선택지:'):
            decisionQuestion = f'**{response}**'
        elif len(response) > 1 and response[1] == '.':  # A., B., C., D. 형식
            choices.append(response.strip())
        else:
            story += response + '\n'

    return {
        'story': story.strip(),
        'decisionQuestion': decisionQuestion,
        'choices': choices if choices else None,
        'dalle_img': dalle_img
    }

# 세션 상태 초기화
def initialize_session():
    for key, default_value in {
        "data_dict": {}, "oid_list": [],
        "openai_api_key": "", "apiBox_state": False,
        "genre_input": "아기 펭귄 보물이의 모험", "genreBox_state": True
    }.items():
        if key not in st.session_state:
            st.session_state[key] = default_value

# OpenAI API Key 인증
def auth():
    st.session_state["apiBox_state"] = True
    st.session_state["genreBox_state"] = False

# 스토리 & 선택지 추가
def add_new_data(story, decisionQuestion, choices, dalle_img):
    oid = str(uuid.uuid4())
    st.session_state['oid_list'].append(oid)
    st.session_state['data_dict'][oid] = (story, decisionQuestion, choices, dalle_img)

# 선택 후 실행 함수
def get_output(_pos: st.empty, oid='', genre=''):
    if not st.session_state.get("openai_api_key"):
        st.warning("OpenAI API Key를 입력하세요!", icon="⚠")
        return

    if oid:
        st.session_state[f'expanded_{oid}'] = False
        st.session_state[f'radio_{oid}_disabled'] = True
        st.session_state[f'submit_{oid}_disabled'] = True
        user_choice = st.session_state[f'radio_{oid}']
    else:
        user_choice = genre
        st.session_state['genreBox_state'] = False

    with _pos:
        data = get_story_and_image(genre, user_choice, st.session_state["openai_api_key"])
        add_new_data(data['story'], data['decisionQuestion'], data['choices'], data['dalle_img'])

# 스토리 생성 UI
def generate_content(story, decisionQuestion, choices, img, oid):
    story_pt = list(st.session_state["data_dict"].keys()).index(oid) + 1
    expander = st.expander(f'Part {story_pt}', expanded=st.session_state.get(f'expanded_{oid}', True))

    col1, col2 = expander.columns([0.65, 0.35])

    if img:
        col2.image(img, use_column_width=True)

    with col1:
        st.write(story)

        if decisionQuestion and choices:
            with st.form(key=f'user_choice_{oid}'):
                st.radio(decisionQuestion, choices, key=f'radio_{oid}', disabled=st.session_state.get(f'radio_{oid}_disabled', False))
                st.form_submit_button("진행하기", disabled=st.session_state.get(f'submit_{oid}_disabled', False), on_click=get_output, args=[st.empty()], kwargs={'oid': oid})

##### 메인 함수 #####
def main():
    initialize_session()
    st.title("📚 NovelGPT")

    # 사이드바
    with st.sidebar:
        st.header("📚 NovelGPT")
        st.info("**Note:** OpenAI API Key를 입력하세요.")
        with st.form(key="API Keys"):
            st.text_input("OpenAI API Key", key="openai_api_key", type="password", disabled=st.session_state["apiBox_state"])
            st.form_submit_button("Submit", on_click=auth)

    # 장르 입력
    col1, col2, col3 = st.columns([8, 1, 1])
    col1.text_input("스토리 주제를 입력하세요", key="genre_input", placeholder="예: 신비한 모험 이야기", disabled=st.session_state["genreBox_state"])
    col3.button("시작!", on_click=get_output, args=[st.empty()], kwargs={'genre': st.session_state["genre_input"]}, disabled=st.session_state["genreBox_state"])

    # 기존 스토리 출력
    for oid in st.session_state["oid_list"]:
        generate_content(*st.session_state["data_dict"][oid], oid)

if __name__ == "__main__":
    main()

2025-02-28 10:21:58.137 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-28 10:21:58.141 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-28 10:21:58.153 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2025-02-28 10:21:58.160 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-28 10:21:58.162 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-28 10:21:58.165 WARNING streamlit.runtime.scriptrunner_utils.script_run_c